In [1]:
# EDEK
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning

You should consider upgrading via the 'C:\Users\Hard-PC\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
from torch import nn

ModuleNotFoundError: No module named 'botocore'

In [ ]:
!mkdir -p dataset

In [ ]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [ ]:
def __iter__(self): return 0

ibm_api_key_id = 'SECRET'
bucket = 'SECRET'

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket=bucket, Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket=bucket, Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket=bucket, Key=key, Filename=filename)

### Model

In [ ]:
get_file('snapshot-uni_df.pkl', 'uni_df.pkl')

In [ ]:
df_onehot = pd.read_pickle('uni_df.pkl')
LABEL_COLUMNS = df_onehot.columns.tolist()[1:]

In [ ]:
LABEL_COLUMNS

In [ ]:
class ArxivAbstracts(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = df.abstract
        self.labels = df[df.columns.difference(['abstract'])]
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        encoding = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=False,
            return_attention_mask= True,
            truncation=True,
            return_tensors='pt'
          )
               
        return {
          'ids': encoding["input_ids"].flatten(),
          'mask': encoding["attention_mask"].flatten(),
          'targets': torch.tensor(self.labels.iloc[item_idx].tolist(), dtype=torch.float)
        }

In [ ]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [ ]:
import pytorch_lightning as pl
class ArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_train, df_test, df_validation, tokenizer, batch_size=16, max_token_len=256):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.df_validation = df_validation
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = ArxivAbstracts(df=self.df_train, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.val_dataset = ArxivAbstracts(df=self.df_validation, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.test_dataset = ArxivAbstracts(df=self.df_test, tokenizer=self.tokenizer, max_len= self.max_token_len)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
arxiv_data = ArxivAbstractsModule(df_train, df_test, df_validation, tokenizer, max_token_len=384)
arxiv_data.setup()

In [ ]:
len(arxiv_data.test_dataloader())/16

In [ ]:
class ArxivAbstractsClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.embedding = nn.Embedding(100000, 60, padding_idx=1)
        self.lstm = nn.LSTM(60, 100, batch_first=True, num_layers=3)
        #self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        #self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classifier = torch.nn.Linear(100, 151) 
        #torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_func = torch.nn.BCEWithLogitsLoss(reduction='mean')
        #self.dropout = torch.nn.Dropout()
        self.save_hyperparameters()
    
    def forward(self, ids, mask, targets=None):
        # roberta layer
#         output = self.pretrained_model(input_ids=ids, attention_mask=mask)
#         pooled_output = torch.mean(output.last_hidden_state, 1)
#         # final logits
#         pooled_output = self.dropout(pooled_output)
#         pooled_output = self.hidden(pooled_output)
#         pooled_output = F.relu(pooled_output)
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
        # calculate loss
        embed = self.embedding(ids)
    
        lengths = torch.sum(mask, axis = 1)
        
        #print(lengths)
            
        packed_embedded = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.tolist(), batch_first=True, enforce_sorted=False)
    
        output, (h_n, c_n)  = self.lstm(packed_embedded)
        
        unpacked, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=1)
        
#         print(unpacked)
        
        logits = self.classifier(unpacked[range(ids.shape[0]), lengths - 1])
        
        loss = 0
        if targets is not None:
            loss = self.loss_func(logits.view(-1, 151), targets.view(-1, 151))
        return loss, logits
  
    def training_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("train_loss", loss.item())
        return {"loss":loss, "predictions": outputs, "labels": batch["targets"]}

    def validation_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("val_loss", loss.item())
        return {"val_loss": loss, "predictions": outputs, "labels": batch["targets"]}

    def predict_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        return torch.sigmoid(outputs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    monitor='val_loss',
    filename='LSTM-{epoch:02d}-{val_loss:.2f}',
    save_top_k=2,
    verbose=True,
    mode='min'
)
logger = TensorBoardLogger("lightning_logs", name="lstm")

config = {
    'model_name': 'distilroberta-base',
    'n_labels': 151,
    'batch_size': 16,
    'lr': 1.5e-6,
    'warmup': 0.2, 
    'train_size': len(arxiv_data.train_dataloader()),
    'weight_decay': 0.001,
    'n_epochs': 30
}

In [ ]:
%%time 
model = ArxivAbstractsClassifier(config)
trainer = Trainer(max_epochs = config['n_epochs'], 
                  gpus = 1, 
                  logger=logger, 
                  enable_checkpointing=True, 
                  callbacks=[checkpoint_callback, RichProgressBar(), EarlyStopping(monitor="val_loss", mode="min")])


trainer.fit(model, datamodule=arxiv_data)

In [ ]:
model_name = "LSTM"
def save_dir_to_cloud(path, model_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                filepath = os.path.join(root,file)
                save_to_cloud(f"outs/{model_name}/{file}", filepath)

In [ ]:
trainer.save_checkpoint(f"{model_name}.ckpt")
save_to_cloud(f'{model_name}.ckpt', f"{model_name}.ckpt")

In [ ]:
save_dir_to_cloud("lightning_logs", model_name)
save_dir_to_cloud("checkpoints", model_name)

In [ ]:
get_file(f'snapshot-{model_name}.ckpt', f"snapshot-{model_name}.ckpt")
# trainer = Trainer()
# trainer.fit(model, datamodule=arxiv_data, ckpt_path=f"snapshot-{model_name}.ckpt")

In [ ]:
model = ArxivAbstractsClassifier(config).load_from_checkpoint(f"snapshot-{model_name}.ckpt", config=config)
trainer = Trainer(gpus = 1, 
                  logger=logger,
                  callbacks=[RichProgressBar()])

In [ ]:
preds = trainer.predict(model, dataloaders=arxiv_data.test_dataloader())

In [ ]:
preds = np.concatenate(preds, axis=0)

In [ ]:
# Probably needs a more sophisticated thingy with model.eval + freezing
# Or just incorporating that to the model above so that loading from a checkpoint will work correctly
def predict(tresh = 0.5):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds]
    y_true = [tensor["targets"] for tensor in arxiv_data.test_dataloader()]
    y_true = np.concatenate([tensor.tolist() for tensor in y_true])
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [ ]:
predict(tresh = 0.35)

In [ ]:
predict(tresh = 0.4)

In [ ]:
predict(tresh = 0.5)

In [ ]:
predict(tresh = 0.2)